In [1]:
from current_season_code.functions import *

In [93]:
def kelly_criterion(row):
    if row['Team1_Prob_Diff'] < 0:
        return 0
    else:
        p = row['Team1_Prob']
        q = 1-p
        ml = row['Home_Odds']
        if ml>=0:
            b = (ml/100)
        if ml<0:
            b = (100/abs(ml))
        kc = ((p*b) - q) / b
        if (kc > 0.5) & (kc<0.6):
            return kc/(kelly+2)
        if (kc > 0.6) & (kc<0.7):
            return kc/(kelly+4)
        if kc > 0.7:
            return kc/(kelly+7)
        else:
            return kc/kelly
def kelly_criterion_2(row):
    if row['Team2_Prob_Diff'] < 0:
        return 0
    else:
        p = row['Team2_Prob']
        q = 1-p
        ml = row['Away_Odds']
        if ml>=0:
            b = (ml/100)
        if ml<0:
            b = (100/abs(ml))
        kc = ((p*b) - q) / b
        if (kc > 0.5) & (kc<0.6):
            return kc/(kelly+2)
        if (kc > 0.6) & (kc<0.7):
            return kc/(kelly+4)
        if kc > 0.7:
            return kc/(kelly+7)
        else:
            return kc/kelly
todays_capital = 100000
ml_param = -1750
ml_param_underdog = 1000
small_advantage = .025
kelly = 12

In [99]:
prob_output = pd.read_csv('current_season_data/todays_stats.csv', index_col = 0)
odds = retrieve_odds(save = False)
odds = odds[['Home_Team', 'Home_Odds', 'Away_Odds', 'Home_Prob', 'Away_Prob']]
merged = pd.merge(prob_output, odds, left_on = 'Team1', right_on = 'Home_Team')
merged.drop(['Home_Team', 'X'], axis = 1, inplace = True)
merged['Team2_Prob'] = 1-merged.Team1_Prob
merged['Team1_Prob_Diff'] = merged.Team1_Prob - merged.Home_Prob/100
merged['Team2_Prob_Diff'] = merged.Team2_Prob - merged.Away_Prob/100
merged['Team1_KC'] = merged.apply(kelly_criterion, axis = 1)
merged['Team2_KC'] = merged.apply(kelly_criterion_2, axis = 1)
merged['Team1_Bet'] = 0
merged['Team2_Bet'] = 0
merged['Team1_Payoff'] = 0
merged['Team2_Payoff'] = 0



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/95.0.4638.69/chromedriver] found in cache


In [102]:
for index, row in merged.iterrows():
    if (row.Team1_KC == 0) & (row.Team2_KC == 0):
        merged.loc[index, 'Team1_Bet'] = 0
        merged.loc[index, 'Team2_Bet'] = 0
        continue
    if ((row.Team1_Prob_Diff<0) & (row.Team2_Prob_Diff<small_advantage)) | ((row.Team1_Prob_Diff<small_advantage) & (row.Team2_Prob_Diff<0)):
        merged.loc[index, 'Team1_Bet'] = 0
        merged.loc[index, 'Team2_Bet'] = 0
        continue
        
    if row.Team1_KC>0:
        merged.loc[index, 'Team1_Bet'] = todays_capital*row.Team1_KC
    if row.Team2_KC>0:
        merged.loc[index, 'Team2_Bet'] = todays_capital*row.Team2_KC
    
    if merged.loc[index, 'Team1_Bet']>0:
        if row.Home_Odds<0:
            merged.loc[index, 'Team1_Payoff'] = (merged.loc[index, 'Team1_Bet']/abs(row.Home_Odds))*100
        if row.Home_Odds>0:
            merged.loc[index, 'Team1_Payoff'] = merged.loc[index, 'Team1_Bet'] * (row.Home_Odds/100)
    if merged.loc[index, 'Team2_Bet']>0:
        if row.Away_Odds<0:
            merged.loc[index, 'Team2_Payoff'] = (merged.loc[index, 'Team2_Bet']/abs(row.Away_Odds))*100
        if row.Away_Odds>0:
            merged.loc[index, 'Team2_Payoff'] = merged.loc[index, 'Team2_Bet'] * (row.Away_Odds/100)
    
    if (merged.loc[index, 'Team1_Bet']>0) & (row.Home_Odds<ml_param):
        merged.loc[index, 'Team1_Bet'] = 0
        merged.loc[index, 'Team1_Payoff'] = 0
        continue
    if (merged.loc[index, 'Team2_Bet']>0) & (row.Away_Odds<ml_param):
        merged.loc[index, 'Team2_Bet'] = 0
        merged.loc[index, 'Team2_Payoff'] = 0
        continue
    if (merged.loc[index, 'Team1_Bet']>0) & (row.Home_Odds>ml_param_underdog):
        merged.loc[index, 'Team1_Bet'] = 0
        merged.loc[index, 'Team1_Payoff'] = 0
        continue
    if (merged.loc[index, 'Team2_Bet']>0) & (row.Away_Odds>ml_param_underdog):
        merged.loc[index, 'Team2_Bet'] = 0
        merged.loc[index, 'Team2_Payoff'] = 0
        continue

In [103]:
merged

,Game_ID,Date,Team1,Team2,Team1_FGM_Per100_Agg,Team1_FGA_Per100_Agg,Team1_FG_PCT_Agg,Team1_FG3M_Per100_Agg,Team1_FG3A_Per100_Agg,Team1_FG3_PCT_Agg,...,Away_Prob,Team2_Prob,Team1_Prob_Diff,Team2_Prob_Diff,Team1_KC,Team2_KC,Team1_Bet,Team2_Bet,Team1_Payoff,Team2_Payoff
0,0,2021-11-11,76ers,Raptors,42.416315,88.872174,0.480083,13.597066,35.822088,0.391083,...,40.650407,0.249592,0.115372,-0.156912,0.026343,0.000000,2634.31804,0.000000,1513.975885,0.000000
1,0,2021-11-11,Jazz,Pacers,40.356324,88.119641,0.459000,13.955361,42.997953,0.324818,...,17.857143,0.141028,-0.002140,-0.037543,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2,0,2021-11-11,Clippers,Heat,38.995000,87.707186,0.447900,13.862997,38.232628,0.364800,...,34.013605,0.417547,-0.119039,0.077411,0.000000,0.009776,0.00000,977.611148,0.000000,1896.565627
